In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm

%matplotlib inline

In [11]:
images_folder = '../../denoising_dataset/temp_checking/raw/'
images_paths = [
    os.path.join(images_folder, fname) 
    for fname in os.listdir(images_folder)
    if fname.endswith('.tiff')
]

images_paths.sort(key=lambda x: int(os.path.splitext(x)[0].split('DSC')[-1]))

images = [
    cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
    for img_path in tqdm(images_paths)
]
images = np.array(images)

  0%|          | 0/140 [00:00<?, ?it/s]

In [20]:
mean_frame = images.astype(np.float16).mean(axis=0).astype(np.uint8)

KeyboardInterrupt: 

In [17]:
diffs_psnr = [
    cv2.ps
    for img in images
]

NameError: name 'a' is not defined